In [1]:
from tensorflow.keras.layers import MaxPooling2D, Conv2D
import numpy as np
import tensorflow as tf

# Step 2 (30 points)
def net48_siso(): 
    height, width = 48, 48
    inputs = tf.keras.Input((height, width, 3));
    x = Conv2D(8,3,1,'same',activation='relu')(inputs)
    # enter your code here
    x = MaxPooling2D(2,2,'valid')(x)
    x = Conv2D(8,3,1,'valid',activation='relu')(x)
    x = MaxPooling2D(2,2,'valid')(x)
    x = Conv2D(4,3,1,'valid',activation='relu')(x)
    x = MaxPooling2D(2,2,'valid')(x)
    x = Conv2D(4,3,1,'valid',activation='relu')(x)
    x = MaxPooling2D(2,2,'valid')(x)
    x = Conv2D(2, 3, 1, 'same', activation='softmax', name='cls_output')(x)
    outputs = x # The output shape of the model is (1, 1, 2)
    return tf.keras.Model(inputs = inputs, outputs = outputs)


In [2]:
# Step 3 (15 points)
def net48_simo(): 
    height, width = 48, 48
    inputs = tf.keras.Input((height, width, 3));
    x_first = Conv2D(8,3,1,'same',activation='relu')(inputs)
    # enter your code here
    def branch_fn(input, final_channels, name, activation=None):
        x = MaxPooling2D(2,2,'valid')(input)
        x = Conv2D(8,3,1,'valid',activation='relu')(x)
        x = MaxPooling2D(2,2,'valid')(x)
        x = Conv2D(4,3,1,'valid',activation='relu')(x)
        x = MaxPooling2D(2,2,'valid')(x)
        x = Conv2D(4,3,1,'valid',activation='relu')(x)
        x = MaxPooling2D(2,2,'valid')(x)
        x = Conv2D(final_channels, 3, 1, 'same', activation=activation, name=name)(x)
        return x
    # for outputs1
    outputs_1 = branch_fn(x_first, 2, 'cls_output', 'softmax') # The output shape of the model is (1, 1, 2)
    # for outputs2
    outputs_2 = branch_fn(x_first, 4, 'reg_output') # The output shape of the model is (1, 1, 4)
    
    # you need to modify the following line
    return tf.keras.Model(inputs = inputs, outputs = (outputs_1, outputs_2))

In [3]:
# Do not change below.
def check_model(cnn_model, is_simo=False):
  print('Checking the CNN model...')
  is_passed = True
  is_cls_output = False
  is_reg_output = False
  ishape = cnn_model.input_shape[1:]
  if (48,48,3) != ishape:
     print('Error! The input shape', ishape, 'is not equal to (48,48,3)')
     is_passed = False
  for i,layer in enumerate(cnn_model.layers):
     if layer.name == 'cls_output':
       oshape = layer.output_shape[1:]
       is_cls_output = True
       if (1,1,2) != oshape:
          print('Error! The output shape', oshape, 'is not equal to (1,1,2)')
          is_passed = False
     if layer.name == 'reg_output':
       oshape = layer.output_shape[1:]
       is_reg_output = True
       if (1,1,4) != oshape:
          print('Error! The output shape', oshape, 'is not equal to (1,1,4)')
          is_passed = False
     layer_type = layer.__class__.__name__
     if layer_type == 'Dense':
       print('Error! Cannot use Dense layers.')
       is_passed = False
     if layer_type == 'Conv2D':
       kernel_size = layer.kernel_size
       strides = layer.strides
       if np.max(kernel_size) > 7:
         print('({:d} {:s}) Error! kernel_size should be less than or equal to 7'.format(i, layer_type))
         is_passed = False
       if np.max(strides) > 2:
         print('({:d} {:s}) Error! stride should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
     if layer_type == 'MaxPooling2D':
       pool_size = layer.pool_size
       strides = layer.strides
       if np.max(pool_size) > 2:
         print('({:d} {:s}) Error! pool_size should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
       if np.max(strides) > 2:
         print('({:d} {:s}) Error! stride should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
  if not is_cls_output:
     print('Error! There should be a Conv. layer with name cls_output')
     is_passed = False
  if is_simo:
     if not is_reg_output:
       print('Error! There should be a Conv. layer with name reg_output')
       is_passed = False
  if is_passed:
     print('Great! The CNN model satisfies all the requirements.')


In [4]:
# Do not change below.
model48_siso = net48_siso()
model48_siso.summary()
check_model(model48_siso)
print()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 8)         224       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 8)         584       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 4)           292   

In [5]:
# Do not change below.
model48_simo = net48_simo()
model48_simo.summary()
check_model(model48_simo, is_simo=True)
print()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 48, 48, 8)    224         ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 24, 24, 8)   0           ['conv2d_4[0][0]']               
                                                                                                  
 max_pooling2d_8 (MaxPooling2D)  (None, 24, 24, 8)   0           ['conv2d_4[0][0]']               
                                                                                            

In [6]:
# Do not change below.
img = np.random.randn(1, 48, 48, 3)
outputs = model48_simo.predict(img)
print(outputs[0].shape) # should display (1,1,1,2)
print(outputs[1].shape) # should display (1,1,1,4)
cls = np.reshape(outputs[0], (2,))
reg = np.reshape(outputs[1], (4,))
print(cls)

1/1 [==============================] - 0s 122ms/step
(1, 1, 1, 2)
(1, 1, 1, 4)
[0.49976736 0.50023264]


In [7]:
print(cls[0] + cls[1])
print(reg)

1.0
[-0.02282757  0.12926647 -0.06428114  0.06944496]


In [8]:
# Question 2 (5 points)
# Why cls[0] + cls[1] is always equal to one regardless of  the values of img?
# Answer:
# The reason is that cls is processed by softmax in the first output from output layer in model48_simo; the softmax function transforms the logits into probabilities in range of 0 to 1, and makes sure that the sum of probabilities is 1.
# Hence, the sum of cls[0] and cls[1] is always equal to one.